<a href="https://colab.research.google.com/github/chriswill88/holbertonschool-machine_learning/blob/master/transfer_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt


TensorFlow 1.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def preprocess_data(X, Y):
  """
    pre-processes the data for your model:

    X is a numpy.ndarray of shape (m, 32, 32, 3) containing the CIFAR 10 data,
      where m is the number of data points
    Y is a numpy.ndarray of shape (m,) containing the CIFAR 10 labels for X
    Returns: X_p, Y_p
    X_p is a numpy.ndarray containing the preprocessed X
    Y_p is a numpy.ndarray containing the preprocessed Y

    one hot incoding for Yww
    resize X for the model
  """
  # one hot y
  encoded_Y = K.utils.to_categorical(Y)

  # Normalize images
  X = X.astype('float32')
  X /= 255

  # rescalining done in model
  return X, encoded_Y

In [4]:
def vgg(inp_t):
  """vgg creates and configure the vgg16 model"""
  i = 0
  out = K.layers.Input(inp_t)
  out = K.layers.Lambda(lambda image: K.backend.resize_images(image, 7, 7, "channels_last"))(out)

  vgg = K.applications.vgg16.VGG16(
      include_top=False, weights='imagenet', input_tensor=out
  )

  output = vgg.layers[-1].output
  output = K.layers.Flatten()(output)
  vgg_model = K.models.Model(vgg.input, output)

  vgg_model.trainable = True
  for layer in vgg_model.layers:
    if i < 11:
      layer.trainable = False
    i += 1

  for i in vgg.layers:
      print(i.name, i.trainable)
  return vgg_model

In [5]:
if __name__ == "__main__":
  ktf = K.backend
  (X_t, Y_t), (X_v, Y_v) = K.datasets.cifar10.load_data()

  X_T, Y_T = preprocess_data(X_t, Y_t)
  X_V, Y_V = preprocess_data(X_v, Y_v)

  inp_t = X_T.shape[1:]
  inp_v = X_V.shape

  vg = vgg(inp_t)
  vg.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
input_1 False
lambda False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool True
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
lambda (Lambda)              (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)

In [6]:
train_datagen = K.preprocessing.image.ImageDataGenerator(zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

val_datagen = K.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow(X_T, Y_T, batch_size=110)
val_generator = val_datagen.flow(X_V, Y_V, batch_size=110)

In [7]:
input_shape = vg.output_shape[1]

model = K.models.Sequential()
model.add(vg)
model.add(K.layers.Dense(512, activation='relu', input_dim=input_shape))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.Dense(512, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=K.optimizers.Adam(lr=2e-5),
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 25088)             14714688  
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 27,828,042
Trainable params: 26,092,554
Non-trainable params: 1,735,488
____________________________________

In [ ]:
history = model.fit_generator(
    train_generator, steps_per_epoch=1000, epochs=20,
    validation_data=val_generator, validation_steps=100,
    verbose=1)         


Epoch 1/20
 180/1000 [====>.........................] - ETA: 10:52 - loss: 2.0205 - acc: 0.2556

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(0, 20)
plt.plot(epochs, loss_train, 'r', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['acc']
acc_val = history.history['val_acc']
epochs = range(0, 20)
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
mood = K.models.load_model('/content/drive/My Drive/cifar10.h5')

model.save('cifar10.h5')

In [ ]:
# fix issue with saving keras applications

K.learning_phase = K.backend.learning_phase
_, (X, Y) = K.datasets.cifar10.load_data()
X_p, Y_p = preprocess_data(X, Y)
mood = K.models.load_model('cifar10.h5')

mood.evaluate(X_p, Y_p, batch_size=128, verbose=1)
